### Integer programming from Discrete Optimization course

* [course](https://www.coursera.org/learn/discrete-optimization/home/welcome) - the course code is here
* [source of code](https://github.com/benalexkeen/Introduction-to-linear-programming/blob/master/Introduction%20to%20Linear%20Programming%20with%20Python%20-%20Part%201.ipynb)
* [another example](https://towardsdatascience.com/linear-programming-and-discrete-optimization-with-python-using-pulp-449f3c5f6e99)



## Small knapsack-problem with only 4 items 


### Read in the data

In [1]:
# this method for reading in the data is obsolete, keeping it in this file for interest
# see next code block for using pandas to read the data

file_location = './discrete_optimization_course/knapsack/data/ks_lecture_dp_2'

with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()
    
from collections import namedtuple
Item = namedtuple("Item", ['index', 'value', 'weight'])

# parse the input
lines = input_data.split('\n')

firstLine = lines[0].split()
item_count = int(firstLine[0])
capacity = int(firstLine[1])

items = []

for i in range(1, item_count+1):
    line = lines[i]
    parts = line.split()
    items.append(Item(i-1, int(parts[0]), int(parts[1])))
    
items


In [116]:
import pandas as pd
file_location = './discrete_optimization_course/knapsack/data/ks_lecture_dp_2'

items = pd.read_csv(file_location,sep=" ",header=None)
items.columns = ["value", "weight"]

no_items = items.iloc[0,0]
capacity = items.iloc[0,1]

items = items.drop(0).reset_index()

In [117]:
items

,index,value,weight
0,1,16,2
1,2,19,3
2,3,23,4
3,4,28,5


### Set up the data for LP

In [133]:
item_names = list(items['index'])
values = dict(zip(item_names,items['value']))
weights = dict(zip(item_names,items['weight']))

In [134]:
weights

{1: 2, 2: 3, 3: 4, 4: 5}

### Set up the model

- the following uses the dictionaries built from the dataframe (second data read commands)

In [122]:
import pulp

In [160]:
# Instantiate a problem class
model = pulp.LpProblem("Small_knapsack_problem", pulp.LpMaximize)

In [161]:
# decision variables - Integer between zero and one
item_include = pulp.LpVariable.dicts("item",
                                     item_names,
                                     lowBound=0,upBound=1,
                                     cat='Integer')

In [162]:
item_include

{1: item_1, 2: item_2, 3: item_3, 4: item_4}

In [163]:
# Objective Function
model += (
    pulp.lpSum([
        values[i]*item_include[i]
        for i in item_names])
)

In [165]:
model += pulp.lpSum(weights[i]*item_include[i] for i in item_names) <= capacity

In [166]:
model

Small_knapsack_problem:
MAXIMIZE
16*item_1 + 19*item_2 + 23*item_3 + 28*item_4 + 0
SUBJECT TO
_C1: 2 item_1 + 3 item_2 + 4 item_3 + 5 item_4 <= 7

VARIABLES
0 <= item_1 <= 1 Integer
0 <= item_2 <= 1 Integer
0 <= item_3 <= 1 Integer
0 <= item_4 <= 1 Integer

In [167]:
model.objective

16*item_1 + 19*item_2 + 23*item_3 + 28*item_4 + 0

In [168]:
model.constraints

OrderedDict([('_C1', 2*item_1 + 3*item_2 + 4*item_3 + 5*item_4 + -7 <= 0)])

In [169]:
# this will write the model to a file
model.writeLP('knapsack_problem')

[item_1, item_2, item_3, item_4]

### Solve the model

In [170]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [171]:
item_include

{1: item_1, 2: item_2, 3: item_3, 4: item_4}

In [156]:
item_include[2].varValue

0.0

### Results

In [172]:
for var in item_include:
    var_value = item_include[var].varValue
    print("Include item {0} : {1}".format(var, var_value) )

Include item 1 : 1.0
Include item 2 : 0.0
Include item 3 : 0.0
Include item 4 : 1.0


In [175]:
total_value_saved = pulp.value(model.objective)

print("The total value saved is €{}".format((total_value_saved)))

The total value saved is €44.0


## Larger knapsack problem

In [ ]:
import pandas as pd
file_location = './discrete_optimization_course/knapsack/data/ks_lecture_dp_2'

items = pd.read_csv(file_location,sep=" ",header=None)
items.columns = ["value", "weight"]

no_items = items.iloc[0,0]
capacity = items.iloc[0,1]

items = items.drop(0).reset_index()